In [2]:
from pathlib import Path
import pandas as pd

DATA_PATH = Path("ev_dataset.csv")
TARGET = "PACK_Q_SOC_INTERNAL"   # 1. 

df = pd.read_csv(DATA_PATH)
print("Ham shape:", df.shape)
df.head(3)


Ham shape: (14658, 38)


,Time_Stamp,latitude,longitude,altitude,PACK_Q_SOC_INTERNAL,PACK_Q_SOC_TRIMMED,CELL_V_MIN_VAL,CELL_V_MAX_VAL,CELL_V_AVG,PACK_V_SUM_OF_CELLS,...,CELL_T_MIN_ID_CMU,CELL_T_MAX_ID_CMU,speeed_set,load,direction,season,cabin_temperature,outdoor_temperature,slope,estimated_charging_time
0,2024-06-26 17:28:33,39.747155,30.474872,821.8,61.290,60.030,3312.34,3319.64,3317.62,49.80,...,0.0,0.0,15,Loaded,CISAR,Summer,NaN,NaN,NaN,NaN
1,2024-06-26 17:28:34,39.747157,30.474876,821.8,61.290,60.030,3311.06,3318.60,3316.50,49.75,...,0.0,0.0,15,Loaded,CISAR,Summer,NaN,NaN,NaN,NaN
2,2024-06-26 17:28:35,39.747153,30.474879,821.8,61.282,60.022,3307.92,3315.39,3313.15,49.70,...,0.0,0.0,15,Loaded,CISAR,Summer,NaN,NaN,NaN,NaN


In [3]:
# Duplikeler
df = df.drop_duplicates()

# Tamamen boş sütunlar
null_all_cols = [c for c in df.columns if df[c].isna().all()]
df = df.drop(columns=null_all_cols)

# Tekil (sabit) sütunlar
nunique = df.nunique(dropna=False)
constant_cols = nunique[nunique <= 1].index.tolist()
df = df.drop(columns=constant_cols)

# Hedef
y = df[TARGET].dropna()
X = df.loc[y.index].drop(columns=[TARGET])

# Sızıntıyı önlemek için diğer SOC sütununu atıldı
for col in ["PACK_Q_SOC_INTERNAL", "PACK_Q_SOC_TRIMMED"]:
    if col in X.columns:
        X = X.drop(columns=col)

print("Final X shape:", X.shape, "y shape:", y.shape)


Final X shape: (14658, 30) y shape: (14658,)


In [4]:
numeric_cols = X.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = [c for c in X.columns if c not in numeric_cols]
print("Sayısal:", len(numeric_cols), " | Kategorik:", len(categorical_cols))


Sayısal: 19  | Kategorik: 11


In [5]:
if numeric_cols:
    corr_series = df[numeric_cols + [TARGET]].corr(numeric_only=True)[TARGET] \
                     .drop(TARGET).sort_values(key=lambda s: s.abs(), ascending=False)
    display(corr_series.to_frame("pearson_corr_to_target").head(20))


,pearson_corr_to_target
cabin_temperature,-0.484211
CELL_V_MAX_VAL,0.459629
CELL_V_AVG,0.441208
PACK_V_SUM_OF_CELLS,0.440970
CELL_V_MIN_VAL,0.417149
DC_Link_Voltage,0.353112
outdoor_temperature,-0.260011
speeed_set,-0.213537
Speed_Measured,-0.167477
Odometer,-0.165277


In [6]:
print("Hedef istatistikleri:")
display(y.describe())

print("İlk 5 sayısal özellik istatistikleri:")
display(df[numeric_cols].describe().T.head())


Hedef istatistikleri:


count    14658.000000
mean        56.615378
std         10.156995
min         34.730000
25%         49.891500
50%         57.460000
75%         65.723250
max         72.770000
Name: PACK_Q_SOC_INTERNAL, dtype: float64

İlk 5 sayısal özellik istatistikleri:


,count,mean,std,min,25%,50%,75%,max
latitude,14658.0,39.749871,0.001766,39.747075,39.748334,39.750004,39.751618,39.752448
longitude,14658.0,30.477740,0.002399,30.474712,30.474912,30.477598,30.480306,30.481239
altitude,14658.0,815.318863,9.000429,793.800000,806.600000,815.000000,823.500000,833.100000
CELL_V_MIN_VAL,14658.0,3259.896965,46.657803,3007.300000,3237.110000,3271.620000,3292.800000,3340.400000
CELL_V_MAX_VAL,14658.0,3270.303775,45.026099,3060.700000,3247.555000,3280.800000,3302.400000,3393.500000


In [8]:
from pathlib import Path
import pandas as pd

DATA_PATH = Path("ev_dataset.csv")
TARGET = "PACK_Q_SOC_TRIMMED"  # 2.

df = pd.read_csv(DATA_PATH)
print("Ham shape:", df.shape)
df.head(3)


Ham shape: (14658, 38)


,Time_Stamp,latitude,longitude,altitude,PACK_Q_SOC_INTERNAL,PACK_Q_SOC_TRIMMED,CELL_V_MIN_VAL,CELL_V_MAX_VAL,CELL_V_AVG,PACK_V_SUM_OF_CELLS,...,CELL_T_MIN_ID_CMU,CELL_T_MAX_ID_CMU,speeed_set,load,direction,season,cabin_temperature,outdoor_temperature,slope,estimated_charging_time
0,2024-06-26 17:28:33,39.747155,30.474872,821.8,61.290,60.030,3312.34,3319.64,3317.62,49.80,...,0.0,0.0,15,Loaded,CISAR,Summer,NaN,NaN,NaN,NaN
1,2024-06-26 17:28:34,39.747157,30.474876,821.8,61.290,60.030,3311.06,3318.60,3316.50,49.75,...,0.0,0.0,15,Loaded,CISAR,Summer,NaN,NaN,NaN,NaN
2,2024-06-26 17:28:35,39.747153,30.474879,821.8,61.282,60.022,3307.92,3315.39,3313.15,49.70,...,0.0,0.0,15,Loaded,CISAR,Summer,NaN,NaN,NaN,NaN


In [9]:
# Duplikeler
df = df.drop_duplicates()

# Tamamen boş sütunlar
null_all_cols = [c for c in df.columns if df[c].isna().all()]
df = df.drop(columns=null_all_cols)

# Tekil (sabit) sütunlar
nunique = df.nunique(dropna=False)
constant_cols = nunique[nunique <= 1].index.tolist()
df = df.drop(columns=constant_cols)

# Hedef
y = df[TARGET].dropna()
X = df.loc[y.index].drop(columns=[TARGET])

# Sızıntıyı önlemek için diğer SOC sütununu atıldı
for col in ["PACK_Q_SOC_INTERNAL", "PACK_Q_SOC_TRIMMED"]:
    if col in X.columns:
        X = X.drop(columns=col)

print("Final X shape:", X.shape, "y shape:", y.shape)


Final X shape: (14658, 30) y shape: (14658,)


In [10]:
numeric_cols = X.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = [c for c in X.columns if c not in numeric_cols]
print("Sayısal:", len(numeric_cols), " | Kategorik:", len(categorical_cols))


Sayısal: 19  | Kategorik: 11


In [11]:
if numeric_cols:
    corr_series = df[numeric_cols + [TARGET]].corr(numeric_only=True)[TARGET] \
                     .drop(TARGET).sort_values(key=lambda s: s.abs(), ascending=False)
    display(corr_series.to_frame("pearson_corr_to_target").head(20))


,pearson_corr_to_target
cabin_temperature,-0.484200
CELL_V_MAX_VAL,0.459636
CELL_V_AVG,0.441214
PACK_V_SUM_OF_CELLS,0.440976
CELL_V_MIN_VAL,0.417155
DC_Link_Voltage,0.353121
outdoor_temperature,-0.259995
speeed_set,-0.213552
Speed_Measured,-0.167475
Odometer,-0.165276


In [12]:
print("Hedef istatistikleri:")
display(y.describe())

print("İlk 5 sayısal özellik istatistikleri:")
display(df[numeric_cols].describe().T.head())


Hedef istatistikleri:


count    14658.000000
mean        55.219026
std         10.461886
min         32.680000
25%         48.291500
50%         56.090000
75%         64.603250
max         71.860000
Name: PACK_Q_SOC_TRIMMED, dtype: float64

İlk 5 sayısal özellik istatistikleri:


,count,mean,std,min,25%,50%,75%,max
latitude,14658.0,39.749871,0.001766,39.747075,39.748334,39.750004,39.751618,39.752448
longitude,14658.0,30.477740,0.002399,30.474712,30.474912,30.477598,30.480306,30.481239
altitude,14658.0,815.318863,9.000429,793.800000,806.600000,815.000000,823.500000,833.100000
CELL_V_MIN_VAL,14658.0,3259.896965,46.657803,3007.300000,3237.110000,3271.620000,3292.800000,3340.400000
CELL_V_MAX_VAL,14658.0,3270.303775,45.026099,3060.700000,3247.555000,3280.800000,3302.400000,3393.500000
